# Train a Resnet on CIFAR10

In [1]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
%matplotlib inline

## Load Dataset

In [2]:
pip install brevitas==0.7.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
from torchvision import transforms
import numpy as np

class GetTransforms():
    '''Returns a list of transformations when type as requested amongst train/test
       Transforms('train') = list of transforms to apply on training data
       Transforms('test') = list of transforms to apply on testing data'''

    def __init__(self):
        pass

    def trainparams(self):
        train_transformations = [ #resises the image so it can be perfect for our model.
            transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
            transforms.RandomRotation((-7,7)),     #Rotates the image to a specified angel
            transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
            transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
            transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261)) #Normalize all the images
            ]

        return train_transformations

    def testparams(self):
        test_transforms = [
            transforms.ToTensor(),
            transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261))
        ]
        return test_transforms

In [4]:
transformations = GetTransforms()
train_transforms = transforms.Compose(transformations.trainparams())
test_transforms = transforms.Compose(transformations.testparams())


class GetCIFAR10_TrainData():
    def __init__(self, dir_name:str):
        self.dirname = dir_name

    def download_train_data(self):
        return datasets.CIFAR10('resnet18/data', train=True, download=True, transform=train_transforms)

    def download_test_data(self):
        return datasets.CIFAR10('resnet18/data', train=False, download=True, transform=test_transforms)

In [5]:
import os
data = GetCIFAR10_TrainData(os.chdir(".."))
trainset = data.download_train_data()
testset = data.download_test_data()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=592,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=592,
                                         shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


## Define a PyTorch Device

GPUs can significantly speed-up training of deep neural networks. We check for availability of a GPU and if so define it as target device.

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

Target device: cpu


## Define the exact MLP Model 

In [7]:
# Setting seeds for reproducibility
torch.manual_seed(0)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, dropout=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.dropout = dropout

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.bn2(self.conv2(out))
        out = F.dropout(out, p=self.dropout)
        out += self.shortcut(x)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=200, dropout=0.0):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.dropout = dropout

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout=self.dropout))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=10, dropout=0.0):
    return ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes, dropout=dropout)

model = model = ResNet18(num_classes=10)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

## Define the Quantized MLP Model 8 bits

We'll now define an MLP model that will be trained to perform inference 

In [8]:
pip install setuptools==69.5.1

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Setting seeds for reproducibility
from brevitas.nn import QuantConv2d, QuantLinear

torch.manual_seed(0)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, dropout=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.dropout = dropout

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.bn2(self.conv2(out))
        out = F.dropout(out, p=self.dropout)
        out += self.shortcut(x)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = QuantConv2d(planes, self.expansion*planes, kernel_size=1, bias=True)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=200, dropout=0.0):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.dropout = dropout

        self.conv1 = QuantConv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = QuantLinear(512*block.expansion, num_classes, bias=True)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout=self.dropout))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=200, dropout=0.0):
    return ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes, dropout=dropout)

model1 = ResNet18(num_classes=10)
model1.to(device)

ResNet(
  (conv1): QuantConv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (output_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (weight_quant): WeightQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (tensor_quant): RescalingIntQuant(
        (int_quant): IntQuant(
          (float_to_int_impl): RoundSte()
          (tensor_clamp_impl): TensorClampSte()
          (delay_wrapper): DelayWrapper(
            (delay_impl): _NoDelay()
          )
        )
        (scaling_impl): StatsFromParameterScaling(
          (parameter_list_stats): _ParameterListStats(
            (first_tracked_param): _ViewParameterWrapper(
              (view_shape_impl): OverTensorView()
            )
            (stats): _Stats(
              (stats_impl): AbsMax()
            )
          )
          (stats_scaling_impl): _St

In [10]:
from brevitas.nn import QuantConv2d, QuantLinear

torch.manual_seed(0)

weight_bit_width = 4

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, dropout=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True, weight_bit_width=weight_bit_width),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.dropout = dropout

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.bn2(self.conv2(out))
        out = F.dropout(out, p=self.dropout)
        out += self.shortcut(x)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = QuantConv2d(planes, self.expansion*planes, kernel_size=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True, weight_bit_width=weight_bit_width),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=200, dropout=0.0):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.dropout = dropout

        self.conv1 = QuantConv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = QuantLinear(512*block.expansion, num_classes, bias=True, weight_bit_width=weight_bit_width)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout=self.dropout))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=10, dropout=0.0):
    return ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes, dropout=dropout)

model2 = ResNet18(num_classes=10)
model2.to(device)

ResNet(
  (conv1): QuantConv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (output_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (weight_quant): WeightQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (tensor_quant): RescalingIntQuant(
        (int_quant): IntQuant(
          (float_to_int_impl): RoundSte()
          (tensor_clamp_impl): TensorClampSte()
          (delay_wrapper): DelayWrapper(
            (delay_impl): _NoDelay()
          )
        )
        (scaling_impl): StatsFromParameterScaling(
          (parameter_list_stats): _ParameterListStats(
            (first_tracked_param): _ViewParameterWrapper(
              (view_shape_impl): OverTensorView()
            )
            (stats): _Stats(
              (stats_impl): AbsMax()
            )
          )
          (stats_scaling_impl): _St

In [11]:
from brevitas.nn import QuantConv2d, QuantLinear

torch.manual_seed(0)

weight_bit_width = 2

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, dropout=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True, weight_bit_width=weight_bit_width),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.dropout = dropout

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.bn2(self.conv2(out))
        out = F.dropout(out, p=self.dropout)
        out += self.shortcut(x)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = QuantConv2d(planes, self.expansion*planes, kernel_size=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True, weight_bit_width=weight_bit_width),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=200, dropout=0.0):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.dropout = dropout

        self.conv1 = QuantConv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True, weight_bit_width=weight_bit_width)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = QuantLinear(512*block.expansion, num_classes, bias=True, weight_bit_width=weight_bit_width)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout=self.dropout))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=10, dropout=0.0):
    return ResNet(BasicBlock, [2,2,2,2], num_classes=num_classes, dropout=dropout)

model3 = ResNet18(num_classes=10)
model3.to(device)

ResNet(
  (conv1): QuantConv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (output_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (weight_quant): WeightQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (tensor_quant): RescalingIntQuant(
        (int_quant): IntQuant(
          (float_to_int_impl): RoundSte()
          (tensor_clamp_impl): TensorClampSte()
          (delay_wrapper): DelayWrapper(
            (delay_impl): _NoDelay()
          )
        )
        (scaling_impl): StatsFromParameterScaling(
          (parameter_list_stats): _ParameterListStats(
            (first_tracked_param): _ViewParameterWrapper(
              (view_shape_impl): OverTensorView()
            )
            (stats): _Stats(
              (stats_impl): AbsMax()
            )
          )
          (stats_scaling_impl): _St

## Train the CNN

We provide two options for training below: you can opt for training the model from scratch (slower) or use a pre-trained model (faster). The first option will give more insight into how the training process works, while the second option will likely give better accuracy.

In [12]:
def train(model, device, train_loader, criterion, optimizer, epoch,
          l1_decay, l2_decay, train_losses, train_accs, scheduler=None):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  avg_loss = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = criterion(y_pred, target)
    if l1_decay > 0:
      l1_loss = 0
      for param in model.parameters():
        l1_loss += torch.norm(param,1)
      loss += l1_decay * l1_loss
    if l2_decay > 0:
      l2_loss = 0
      for param in model.parameters():
        l2_loss += torch.norm(param,2)
      loss += l2_decay * l2_loss

    # Backpropagation
    loss.backward()
    optimizer.step()
    if scheduler:
      scheduler.step()

    # Update pbar-tqdm
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    avg_loss += loss.item()

    pbar_str = f'Loss={loss.item():0.5f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}'
    if l1_decay > 0:
      pbar_str = f'L1_loss={l1_loss.item():0.3f} %s' % (pbar_str)
    if l2_decay > 0:
      pbar_str = f'L2_loss={l2_loss.item():0.3f} %s' % (pbar_str)

    pbar.set_description(desc= pbar_str)

  avg_loss /= len(train_loader)
  avg_acc = 100*correct/processed
  train_accs.append(avg_acc)
  train_losses.append(avg_loss)

## Test the CNN 

In [13]:
classes = ["%s" % i for i in range(10)]

In [14]:
def test(model, device, test_loader, criterion, classes, test_losses, test_accs,
         misclassified_imgs, correct_imgs, is_last_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss +=criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            is_correct = pred.eq(target.view_as(pred))
            if is_last_epoch:
              misclassified_inds = (is_correct==0).nonzero()[:,0]
              for mis_ind in misclassified_inds:
                if len(misclassified_imgs) == 25:
                  break
                misclassified_imgs.append({
                    "target": target[mis_ind].cpu().numpy(),
                    "pred": pred[mis_ind][0].cpu().numpy(),
                    "img": data[mis_ind]
                })
              
              correct_inds = (is_correct==1).nonzero()[:,0]
              for ind in correct_inds:
                if len(correct_imgs) == 25:
                  break
                correct_imgs.append({
                    "target": target[ind].cpu().numpy(),
                    "pred": pred[ind][0].cpu().numpy(),
                    "img": data[ind]
                })
            correct += is_correct.sum().item()

    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    test_acc = 100. * correct / len(test_loader.dataset)
    test_accs.append(test_acc)

    if test_acc >= 90.0:
        classwise_acc(model, device, test_loader, classes)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_acc))


In [15]:
def classwise_acc(model, device, test_loader, classes):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    # print class-wise test accuracies
    print()
    for i in range(10):
      print('Accuracy of %5s : %2d %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))
    print()

## Train and Test

In [16]:
import os
import torch

# Make sure the model is on CPU before loading a pretrained state_dict
model = model.cpu()
model1 = model1.cpu()


In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum= 0.9)
test_losses, train_losses, test_accs, train_accs = [], [], [], []
misclassified_imgs, correct_imgs = [], []

In [18]:
# Save the Brevitas model to disk
trained_state_dict = torch.load("CIFAR/models/resnet18.pth")
model.load_state_dict(trained_state_dict, strict=False)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# Save the Brevitas model to disk
trained_state_dict = torch.load("CIFAR/models/quentresnet18_weight8.pth")
model1.load_state_dict(trained_state_dict, strict=False)

In [ ]:
# Save the Brevitas model to disk
trained_state_dict = torch.load("CIFAR/models/quentresnet18_weight4.pth")
model2.load_state_dict(trained_state_dict, strict=False)

In [ ]:
# Save the Brevitas model to disk
trained_state_dict = torch.load("CIFAR/models/quentresnet18_weight2.pth")
model3.load_state_dict(trained_state_dict, strict=False)

In [ ]:
# Move the model back to it's target device
model.to(device)
# Test for accuracy
exact_acc = test(model, device, testloader, criterion, classes, test_losses,
                 test_accs, misclassified_imgs, correct_imgs,False)

In [ ]:
# Move the model back to it's target device
model1.to(device)
# Test for accuracy
exact_acc = test(model1, device, testloader, criterion, classes, test_losses,
                 test_accs, misclassified_imgs, correct_imgs,False)

In [ ]:
# Move the model back to it's target device
model2.to(device)
# Test for accuracy
exact_acc = test(model2, device, testloader, criterion, classes, test_losses,
                 test_accs, misclassified_imgs, correct_imgs,False)

In [ ]:
# Move the model back to it's target device
model3.to(device)
# Test for accuracy
exact_acc = test(model3, device, testloader, criterion, classes, test_losses,
                 test_accs, misclassified_imgs, correct_imgs,False)

In [ ]:
!pip install torch torchattacks tqdm

## Ataque FGSM

In [ ]:
def attack_test(model, device, test_loader, criterion, classes, test_losses, test_accs,
         misclassified_imgs, correct_imgs, is_last_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        adv_images = atk(data, output)
        with torch.no_grad():
            output = model(adv_images)
            test_loss +=criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            is_correct = pred.eq(target.view_as(pred))
            if is_last_epoch:
              misclassified_inds = (is_correct==0).nonzero()[:,0]
              for mis_ind in misclassified_inds:
                if len(misclassified_imgs) == 25:
                  break
                misclassified_imgs.append({
                    "target": target[mis_ind].cpu().numpy(),
                    "pred": pred[mis_ind][0].cpu().numpy(),
                    "img": data[mis_ind]
                })
              
              correct_inds = (is_correct==1).nonzero()[:,0]
              for ind in correct_inds:
                if len(correct_imgs) == 25:
                  break
                correct_imgs.append({
                    "target": target[ind].cpu().numpy(),
                    "pred": pred[ind][0].cpu().numpy(),
                    "img": data[ind]
                })
            correct += is_correct.sum().item()

    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    test_acc = 100. * correct / len(test_loader.dataset)
    test_accs.append(test_acc)

    if test_acc >= 90.0:
        classwise_acc(model, device, test_loader, classes)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_acc))
    return test_acc

In [ ]:
from torchattacks import FGSM
atk = FGSM(model1, eps=0.3)
print(atk)

In [ ]:
fgsm_eps = 0
fgsm_accs1 = []
eps = []
while fgsm_eps <= 0.3:
    atk = FGSM(model1, fgsm_eps)
    fgsm_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    fgsm_accs1.append(fgsm_acc1)
    eps.append(fgsm_eps)
    fgsm_eps += 0.003

In [ ]:
fgsm_eps = 0
fgsm_accs = []
eps = []
while fgsm_eps <= 0.3:
    atk = FGSM(model, fgsm_eps)
    fgsm_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    fgsm_accs.append(fgsm_acc)
    eps.append(fgsm_eps)
    fgsm_eps += 0.003

In [ ]:
fgsm_eps = 0
fgsm_accs2 = []
eps = []
while fgsm_eps <= 0.3:
    atk = FGSM(model2, fgsm_eps)
    fgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    fgsm_accs2.append(fgsm_acc2)
    eps.append(fgsm_eps)
    fgsm_eps += 0.003

In [ ]:
fgsm_eps = 0
fgsm_accs3 = []
eps = []
while fgsm_eps <= 0.3:
    atk = FGSM(model3, fgsm_eps)
    fgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    fgsm_accs3.append(fgsm_acc3)
    eps.append(fgsm_eps)
    fgsm_eps += 0.003

## Graficar

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array(eps)
ypoints = np.array(fgsm_accs)
y1points = np.array(fgsm_accs1)
y2points = np.array(fgsm_accs2)
y3points = np.array(fgsm_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

# Ataque BIM

In [ ]:
from torchattacks import BIM
atk = BIM(model, eps=0.3, alpha=0.2, steps=50)
print(atk)

In [ ]:
bim_eps = 0
bim_accs1 = []
eps = []
while bim_eps <= 0.3:
    atk = BIM(model1, bim_eps, alpha=0.2, steps=50)
    bim_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    bim_accs1.append(bim_acc1)
    eps.append(bim_eps)
    bim_eps += 0.003

In [ ]:
bim_eps = 0
bim_accs = []
eps = []
while bim_eps <= 0.3:
    atk = BIM(model, bim_eps, alpha=0.2, steps=50)
    bim_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    bim_accs.append(bim_acc)
    eps.append(bim_eps)
    bim_eps += 0.003

In [ ]:
bim_eps = 0
bim_accs2 = []
eps = []
while bim_eps <= 0.3:
    atk = BIM(model2, bim_eps, alpha=0.2, steps=50)
    bim_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    bim_accs2.append(bim_acc2)
    eps.append(bim_eps)
    bim_eps += 0.003

In [ ]:
bim_eps = 0
bim_accs3 = []
eps = []
while bim_eps <= 0.3:
    atk = BIM(model3, bim_eps, alpha=0.2, steps=50)
    bim_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    bim_accs3.append(bim_acc3)
    eps.append(bim_eps)
    bim_eps += 0.003

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array(eps)
ypoints = np.array(bim_accs)
y1points = np.array(bim_accs1)
y2points = np.array(bim_accs2)
y3points = np.array(bim_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque RFGSM

In [ ]:
from torchattacks import RFGSM
atk = RFGSM(model, eps=0.3, alpha=0.01, steps=40)
print(atk)

In [ ]:
rfgsm_eps = 0
rfgsm_accs = []
eps = []
while rfgsm_eps <= 0.3:
    atk = RFGSM(model, rfgsm_eps, alpha=0.2, steps=50)
    rfgsm_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    rfgsm_accs.append(rfgsm_acc)
    eps.append(rfgsm_eps)
    rfgsm_eps += 0.003

In [ ]:
rfgsm_eps1 = 0
rfgsm_accs1 = []
eps = []
while rfgsm_eps1 <= 0.3:
    atk = RFGSM(model1, rfgsm_eps1, alpha=0.2, steps=50)
    rfgsm_acc1 = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    rfgsm_accs1.append(rfgsm_acc1)
    eps.append(rfgsm_eps1)
    rfgsm_eps1 += 0.003

In [ ]:
rfgsm_eps2 = 0
rfgsm_accs2 = []
eps = []
while rfgsm_eps2 <= 0.3:
    atk = RFGSM(model2, rfgsm_eps2, alpha=0.2, steps=50)
    rfgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    rfgsm_accs2.append(rfgsm_acc2)
    eps.append(rfgsm_eps2)
    rfgsm_eps2 += 0.003

In [ ]:
rfgsm_eps3 = 0
rfgsm_accs3 = []
eps = []
while rfgsm_eps3 <= 0.3:
    atk = RFGSM(model3, rfgsm_eps3, alpha=0.2, steps=50)
    rfgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    rfgsm_accs3.append(rfgsm_acc3)
    eps.append(rfgsm_eps3)
    rfgsm_eps3 += 0.003

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array(eps)
ypoints = np.array(rfgsm_accs)
y1points = np.array(rfgsm_accs1)
y2points = np.array(rfgsm_accs2)
y3points = np.array(rfgsm_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque PGD

In [ ]:
from torchattacks import PGD
atk = PGD(model, eps=0.3, alpha=0.01, steps=40, random_start=True)
print(atk)

In [ ]:
pgd_eps = 0
pgd_accs = []
eps = []
while pgd_eps <= 0.3:
    atk = PGD(model, pgd_eps, alpha=0.2, steps=50, random_start=True)
    pgd_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    pgd_accs.append(pgd_acc)
    eps.append(pgd_eps)
    pgd_eps += 0.003

In [ ]:
pgd_eps1 = 0
pgd_accs1 = []
eps1 = []
while pgd_eps1 <= 0.3:
    atk = PGD(model1, pgd_eps, alpha=0.2, steps=50, random_start=True)
    pgd_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    pgd_accs1.append(pgd_acc1)
    eps1.append(pgd_eps1)
    pgd_eps1 += 0.003

In [ ]:
pgd_eps2 = 0
pgd_accs2 = []
eps2 = []
while pgd_eps2 <= 0.3:
    atk = PGD(model2, pgd_eps, alpha=0.2, steps=50, random_start=True)
    pgd_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    pgd_accs2.append(pgd_acc2)
    eps2.append(pgd_eps2)
    pgd_eps2 += 0.003

In [ ]:
pgd_eps3 = 0
pgd_accs3 = []
eps3 = []
while pgd_eps3 <= 0.3:
    atk = PGD(model3, pgd_eps, alpha=0.2, steps=50, random_start=True)
    pgd_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    pgd_accs3.append(pgd_acc3)
    eps3.append(pgd_eps3)
    pgd_eps3 += 0.003

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array(eps)
ypoints = np.array(pgd_accs)
y1points = np.array(pgd_accs1)
y2points = np.array(pgd_accs2)
y3points = np.array(pgd_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, '--',label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque EOTPGD

In [ ]:
from torchattacks import EOTPGD
atk = EOTPGD(model, eps=0.3, alpha=0.01, steps=40, eot_iter=2)
print(atk)

In [ ]:
pgd_eps = 0
EOTpgd_accs = []
eps = []
while pgd_eps <= 0.3:
    atk =EOTPGD(model, pgd_eps, alpha=0.01, steps=40, eot_iter=2)
    pgd_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    EOTpgd_accs.append(pgd_acc)
    eps.append(pgd_eps)
    pgd_eps += 0.003

In [ ]:
pgd_eps1 = 0
EOTpgd_accs1 = []
eps1 = []
while pgd_eps1 <= 0.3:
    atk = EOTPGD(model1, pgd_eps, alpha=0.01, steps=40, eot_iter=2)
    pgd_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    EOTpgd_accs1.append(pgd_acc1)
    eps1.append(pgd_eps1)
    pgd_eps1 += 0.003

In [ ]:
pgd_eps2 = 0
EOTpgd_accs2 = []
eps2 = []
while pgd_eps2 <= 0.3:
    atk = EOTPGD(model2, pgd_eps, alpha=0.01, steps=40, eot_iter=2)
    pgd_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    EOTpgd_accs2.append(pgd_acc2)
    eps2.append(pgd_eps2)
    pgd_eps2 += 0.003

In [ ]:
pgd_eps3 = 0
EOTpgd_accs3 = []
eps3 = []
while pgd_eps3 <= 0.3:
    atk = EOTPGD(model3, pgd_eps, alpha=0.01, steps=40, eot_iter=2)
    pgd_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    EOTpgd_accs3.append(pgd_acc3)
    eps3.append(pgd_eps3)
    pgd_eps3 += 0.003

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array(eps)
ypoints = np.array(EOTpgd_accs)
y1points = np.array(EOTpgd_accs1)
y2points = np.array(EOTpgd_accs2)
y3points = np.array(EOTpgd_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, '--',label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

# Ataque FFGSM¶

In [ ]:
from torchattacks import FFGSM
atk = FFGSM(model, eps=0.3, alpha=0.01)
print(atk)

In [ ]:
ffgsm_eps = 0
ffgsm_accs1 = []
eps = []
while ffgsm_eps <= 0.3:
    atk = FFGSM(model1, ffgsm_eps, alpha=0.01)
    ffgsm_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    ffgsm_accs1.append(ffgsm_acc1)
    eps.append(ffgsm_eps)
    ffgsm_eps += 0.003

In [ ]:
ffgsm_eps = 0
ffgsm_accs = []
eps = []
while ffgsm_eps <= 0.3:
    atk = FFGSM(model, ffgsm_eps, alpha=0.01)
    ffgsm_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    ffgsm_accs.append(ffgsm_acc)
    eps.append(ffgsm_eps)
    ffgsm_eps += 0.003

In [ ]:
ffgsm_eps = 0
ffgsm_accs2 = []
eps = []
while ffgsm_eps <= 0.3:
    atk = FFGSM(model2, ffgsm_eps, alpha=0.01)
    ffgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    ffgsm_accs2.append(ffgsm_acc2)
    eps.append(ffgsm_eps)
    ffgsm_eps += 0.003

In [ ]:
ffgsm_eps = 0
ffgsm_accs3 = []
eps = []
while ffgsm_eps <= 0.3:
    atk = FFGSM(model3, ffgsm_eps, alpha=0.01)
    ffgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    ffgsm_accs3.append(ffgsm_acc3)
    eps.append(ffgsm_eps)
    ffgsm_eps += 0.003

In [ ]:
xpoints = np.array(eps)
ypoints = np.array(ffgsm_accs)
y1points = np.array(ffgsm_accs1)
y2points = np.array(ffgsm_accs2)
y3points = np.array(ffgsm_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque TPGD

In [ ]:
from torchattacks import TPGD
atk = TPGD(model, eps=0.3, alpha=0.01, steps=40)
print(atk)

In [ ]:
tpgd_eps = 0
tpgd_accs = []
eps = []
while tpgd_eps <= 0.3:
    atk = TPGD(model, tpgd_eps, alpha=0.2, steps=40)
    tpgd_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tpgd_accs.append(tpgd_acc)
    eps.append(tpgd_eps)
    tpgd_eps += 0.003

In [ ]:
tpgd_eps1 = 0
tpgd_accs1 = []
eps1 = []
while tpgd_eps1 <= 0.3:
    atk = TPGD(model1, tpgd_eps1, alpha=0.2, steps=40)
    tpgd_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tpgd_accs1.append(tpgd_acc1)
    eps1.append(tpgd_eps1)
    tpgd_eps1 += 0.003

In [ ]:
tpgd_eps2 = 0
tpgd_accs2 = []
eps2 = []
while tpgd_eps2 <= 0.3:
    atk = TPGD(model2, tpgd_eps2, alpha=0.2, steps=40)
    tpgd_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tpgd_accs2.append(tpgd_acc2)
    eps2.append(tpgd_eps2)
    tpgd_eps2 += 0.003

In [ ]:
tpgd_eps3 = 0
tpgd_accs3 = []
eps3 = []
while tpgd_eps3 <= 0.3:
    atk = TPGD(model3, tpgd_eps3, alpha=0.2, steps=40)
    tpgd_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tpgd_accs3.append(tpgd_acc3)
    eps3.append(tpgd_eps3)
    tpgd_eps3 += 0.003

In [ ]:
xpoints = np.array(eps)
ypoints = np.array(tpgd_accs)
y1points = np.array(tpgd_accs1)
y2points = np.array(tpgd_accs2)
y3points = np.array(tpgd_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque MIFSGM

In [ ]:
from torchattacks import MIFGSM
atk = MIFGSM(model, eps=0.3, steps=40, decay=1.0)
print(atk)

In [ ]:
mifgsm_eps = 0
mifgsm_accs = []
eps = []
while mifgsm_eps <= 0.3:
    atk = MIFGSM(model, mifgsm_eps, steps=40, decay=1.0)
    mifgsm_acc1 = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    mifgsm_accs.append(mifgsm_acc1)
    eps.append(mifgsm_eps)
    mifgsm_eps += 0.003

In [ ]:
mifgsm_eps = 0
mifgsm_accs1 = []
eps = []
while mifgsm_eps <= 0.3:
    atk = MIFGSM(model1, eps=mifgsm_eps, steps=40, decay=1.0)
    mifgsm_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    mifgsm_accs1.append(mifgsm_acc1)
    eps.append(mifgsm_eps)
    mifgsm_eps += 0.003

In [ ]:
mifgsm_eps = 0
mifgsm_accs2 = []
eps = []
while mifgsm_eps <= 0.3:
    atk = MIFGSM(model2, mifgsm_eps, steps=40, decay=1.0)
    mifgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    mifgsm_accs2.append(mifgsm_acc2)
    eps.append(mifgsm_eps)
    mifgsm_eps += 0.003

In [ ]:
mifgsm_eps = 0
mifgsm_accs3 = []
eps = []
while mifgsm_eps <= 0.3:
    atk = MIFGSM(model3, mifgsm_eps, steps=40, decay=1.0)
    mifgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    mifgsm_accs3.append(mifgsm_acc3)
    eps.append(mifgsm_eps)
    mifgsm_eps += 0.003

In [ ]:
import matplotlib.pyplot as plt
xpoints = np.array(eps)
ypoints = np.array(mifgsm_accs)
y1points = np.array(mifgsm_accs1)
y2points = np.array(mifgsm_accs2)
y3points = np.array(mifgsm_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

## Ataque UPGD

In [ ]:
from torchattacks import UPGD
atk = UPGD(model, eps=0.3, steps=40, random_start=False)
print(atk)

In [ ]:
upgd_eps2 = 0
upgd_accs2 = []
eps2 = []
while upgd_eps2 <= 0.3:
    atk = UPGD(model2, upgd_eps2, alpha=0.2, steps=40)
    upgd_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    upgd_accs2.append(upgd_acc2)
    eps2.append(upgd_eps2)
    upgd_eps2 += 0.003

In [ ]:
upgd_eps = 0
upgd_accs = []
eps = []
while upgd_eps <= 0.3:
    atk = UPGD(model, upgd_eps,steps=40, random_start=False)
    upgd_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    upgd_accs.append(upgd_acc)
    eps.append(upgd_eps)
    upgd_eps += 0.003

In [ ]:
upgd_eps1 = 0
upgd_accs1 = []
eps1 = []
while upgd_eps1 <= 0.3:
    atk = UPGD(model1, upgd_eps1, alpha=0.2, steps=40)
    upgd_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    upgd_accs1.append(upgd_acc1)
    eps1.append(upgd_eps1)
    upgd_eps1 += 0.003

In [ ]:
upgd_eps3 = 0
upgd_accs3 = []
eps3 = []
while upgd_eps3 <= 0.3:
    atk = UPGD(model3, upgd_eps3, alpha=0.2, steps=40)
    upgd_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    upgd_accs3.append(upgd_acc3)
    eps3.append(upgd_eps3)
    upgd_eps3 += 0.003

In [ ]:
xpoints = np.array(eps2)
ypoints = np.array(upgd_accs)
y1points = np.array(upgd_accs1)
y2points = np.array(upgd_accs2)
y3points = np.array(upgd_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

# Ataque DIFGSM

In [ ]:
from torchattacks import DIFGSM
atk = DIFGSM(model, eps=0.3, alpha=0.01, steps=40, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
print(atk)

In [ ]:
difgsm_eps = 0
difgsm_accs = []
eps = []
while difgsm_eps <= 0.3:
    atk = DIFGSM(model, difgsm_eps, alpha=0.01, steps=40, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
    difgsm_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    difgsm_accs.append(difgsm_acc)
    eps.append(mifgsm_eps)
    difgsm_eps += 0.003

In [ ]:
difgsm_eps1 = 0
difgsm_accs1 = []
eps = []
while difgsm_eps1 <= 0.3:
    atk = DIFGSM(model1, difgsm_eps1, alpha=0.01, steps=40, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
    difgsm_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    difgsm_accs1.append(difgsm_acc1)
    eps.append(difgsm_eps1)
    difgsm_eps1 += 0.003

In [ ]:
difgsm_eps2 = 0
difgsm_accs2 = []
eps = []
while difgsm_eps2 <= 0.3:
    atk = DIFGSM(model2, difgsm_eps2, alpha=0.01, steps=40, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
    difgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    difgsm_accs2.append(difgsm_acc2)
    eps.append(difgsm_eps2)
    difgsm_eps2 += 0.003

In [ ]:
difgsm_eps3 = 0
difgsm_accs3 = []
eps = []
while difgsm_eps3 <= 0.3:
    atk = DIFGSM(model3, difgsm_eps3, alpha=0.01, steps=40, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
    difgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    difgsm_accs3.append(difgsm_acc3)
    eps.append(difgsm_eps3)
    difgsm_eps3 += 0.003

In [ ]:
xpoints = np.array(eps)
ypoints = np.array(difgsm_accs)
y1points = np.array(difgsm_accs1)
y2points = np.array(difgsm_accs2)
y3points = np.array(difgsm_accs3)

plt.plot(xpoints, ypoints, label = "Red exacta")
plt.plot(xpoints, y1points, label = "Cuantización 1")
plt.plot(xpoints, y2points, label = "Cuantización 2")
plt.plot(xpoints, y3points, label = "Cuantización 3")
#plt.title("FGSM Attack")
plt.xlabel("Perturbación máxima (eps)")
plt.ylabel("Exactitud (%)")
plt.legend(loc="best")
plt.show()

# Ataque TIFGSM

In [ ]:
from torchattacks import TIFGSM
atk = TIFGSM(model, eps=0.3, alpha=0.01, steps=40, decay=1.0, resize_rate=0.9, diversity_prob=0.7, random_start=False)
print(atk)

In [ ]:
tifgsm_eps = 0
tifgsm_accs = []
eps = []
while tifgsm_eps <= 0.3:
    atk = TIFGSM(model, tifgsm_eps, alpha=0.01, steps=40, decay=1.0, resize_rate=0.9, diversity_prob=0.7, random_start=False)
    tifgsm_acc = attack_test(model, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tifgsm_accs.append(tifgsm_acc)
    eps.append(tifgsm_eps)
    tifgsm_eps += 0.003

In [ ]:
tifgsm_eps1 = 0
tifgsm_accs1 = []
eps = []
while tifgsm_eps1<= 0.3:
    atk = TIFGSM(model1, tifgsm_eps1, alpha=0.01, steps=40, decay=1.0, resize_rate=0.9, diversity_prob=0.7, random_start=False)
    tifgsm_acc1 = attack_test(model1, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tifgsm_accs1.append(tifgsm_acc1)
    eps.append(tifgsm_eps1)
    tifgsm_eps1 += 0.003

In [ ]:
tifgsm_eps2 = 0
tifgsm_accs2 = []
eps = []
while tifgsm_eps2<= 0.3:
    atk = TIFGSM(model2, tifgsm_eps2, alpha=0.01, steps=40, decay=1.0, resize_rate=0.9, diversity_prob=0.7, random_start=False)
    tifgsm_acc2 = attack_test(model2, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tifgsm_accs2.append(tifgsm_acc2)
    eps.append(tifgsm_eps2)
    tifgsm_eps2 += 0.003

In [ ]:
tifgsm_eps3 = 0
tifgsm_accs3 = []
eps = []
while tifgsm_eps3<= 0.3:
    atk = TIFGSM(model3, tifgsm_eps3, alpha=0.01, steps=40, decay=1.0, resize_rate=0.9, diversity_prob=0.7, random_start=False)
    tifgsm_acc3 = attack_test(model3, device, testloader, criterion, classes, test_losses,
                       test_accs, misclassified_imgs, correct_imgs, False)    
    tifgsm_accs3.append(tifgsm_acc3)
    eps.append(tifgsm_eps3)
    tifgsm_eps3 += 0.003